In [35]:
!python pipeline.py

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
/opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-us-east-1-975049894496/DoomTrainingPipeline/code/0e42c63414499d92db5451321cb2380aeacd3dbfa6d4382a431835b6ffd53356/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-975049894496/DoomTrain

In [1]:
import boto3
sm = boto3.client("sagemaker")
PIPELINE_NAME = "DoomTrainingPipeline"

execs = sm.list_pipeline_executions(PipelineName=PIPELINE_NAME, MaxResults=3)["PipelineExecutionSummaries"]
assert execs, "No executions found"
last = execs[0]
print("Execution:", last["PipelineExecutionArn"], last["PipelineExecutionStatus"])

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=last["PipelineExecutionArn"])["PipelineExecutionSteps"]
for s in steps:
    print(s["StepName"], s["StepStatus"], s.get("FailureReason"))


Execution: arn:aws:sagemaker:us-east-1:975049894496:pipeline/DoomTrainingPipeline/execution/lzvwig82xwwe Failed
DoomBatchTransform Failed ClientError: ClientError: API error (400): OCI runtime create failed: runc create failed: unable to start container process: error during container init: exec: "serve": executable file not found in $PATH: unknown
DoomCreateModel-CreateModel Succeeded None
DoomRegisterModel-RegisterModel Succeeded None
QualityGate Succeeded None
DoomEvaluate Succeeded None
DoomTrain Succeeded None
DoomPreprocess Succeeded None


In [4]:
# Find eval job for this execution
eval_step = next(s for s in steps if s["StepName"] == "DoomEvaluate")
eval_job = eval_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
desc = sm.describe_processing_job(ProcessingJobName=eval_job)

eval_s3 = next(o["S3Output"]["S3Uri"] for o in desc["ProcessingOutputConfig"]["Outputs"] if o["OutputName"]=="evaluation")
print("Eval S3:", eval_s3)

import boto3, json
s3 = boto3.client("s3")
bkt, key = eval_s3.replace("s3://","").split("/",1)
obj = s3.get_object(Bucket=bkt, Key=key.rstrip("/") + "/evaluation.json")
report = json.loads(obj["Body"].read())
f1 = report["binary_classification_metrics"]["macro_avg"]["f1"]
print("F1 in artifact:", f1, type(f1))


Eval S3: s3://sagemaker-us-east-1-975049894496/DoomTrainingPipeline/lzvwig82xwwe/DoomEvaluate/output/evaluation
F1 in artifact: 0.8940644850839616 <class 'float'>
